In [3]:
#allows any changes on external files to be loaded on code run.
%load_ext autoreload
%autoreload 2

In [23]:
from shine_ai.dataset import DataSet
from shine_ai.parser import Parser
import os.path as path


import scipy.io
import pandas as pd

mat = scipy.io.loadmat('data/data_ST.mat')

data_path = path.join('.','data','data_ST.mat')
parser = Parser(file_path='./data/data_ST.mat')

data_sets = []

for index, exp_lbl in enumerate(parser.experiment_labels):
    set = DataSet(name=exp_lbl, parser=parser, experiment_index=index)
    data_sets.append(set)

data_set1 = data_sets[0]

print(data_set1.x[0:5])




values (4, 576) [[5.96831037e+18 5.96831037e+18 5.96831037e+18 ... 5.07306381e+19
  5.07306381e+19 5.07306381e+19]
 [1.00000000e+00 1.00000000e+00 1.00000000e+00 ... 1.50000000e+00
  1.50000000e+00 1.50000000e+00]
 [5.00000000e+02 5.00000000e+02 6.00000000e+02 ... 9.00000000e+02
  1.00000000e+03 1.00000000e+03]
 [0.00000000e+00 1.00000000e+00 0.00000000e+00 ... 1.00000000e+00
  0.00000000e+00 1.00000000e+00]]


AxisError: axis 1 is out of bounds for array of dimension 1

576


In [191]:
experiment_label = [str(i[0]) for i in (mat['data'][0][0][3][0][0][0])]
experiment_label

['Hplasma_HNBI',
 'Dplasma_HNBI',
 'Dplasma_DNBI',
 'DTplasma_HNBI',
 'DTplasma_DNBI']

In [113]:
inputs_labels = [str(i[0]) for i in mat['data'][0][0][0][0][1:]]
inputs_labels

inputs = {}

for i, label in enumerate(inputs_labels, start=1):
    inputs[label] = mat['data'][0][0][3][0][i][0]
inputs






{'line_av_density': array([5.96831037e+18, 1.23629286e+19, 1.87575469e+19, 2.51521651e+19,
        3.15467834e+19, 3.79414016e+19, 4.43360199e+19, 5.07306381e+19]),
 'density_peaking': array([1. , 1.1, 1.2, 1.3, 1.4, 1.5]),
 'E_NBI': array([ 500,  600,  700,  800,  900, 1000], dtype=uint16),
 'injector_number': array([array(['off-axis'], dtype='<U8'), array(['on-axis'], dtype='<U7')],
       dtype=object)}

In [71]:
df = pd.DataFrame()
df.(mat['data'][0][0][0][0])

TypeError: 'RangeIndex' object is not callable